# Issues with lodash-fp

`lodash-fp` is nice but some functions do not work as well as they did in lodash itself.

Created to accompany the [bug report 1343](https://github.com/lodash/lodash/issues/1343).

In [2]:
var _ = require('lodash-fp');
var lodash = require('lodash');

undefined

### No way to provide the default value to `result`?!

In lodash, `_.result({}, "noSuchProperty", "default")` returns `"default"`.
In lodash-fp we put the object to the end but there is no (clear) way to provide the default param:

In [3]:
[_.result("default", "noSuchProperty", {}), 
 _.result("noSuchProperty", "default", {}), 
 _.result("noSuchProperty", {}, "default")]

[ undefined, undefined, undefined, undefined ]

### `assign` behaves strangely

The correct way to call `assign` in lodash-fp is with the object being changed in the last position:

In [5]:
_.assign({a:1}, {b:2}, {}); // => {a:1, b:2} as expected

{ b: 2, a: 1 }

However if we by a chance put an empty object to the first place, we get something weird:

In [7]:
_.assign({}, {a:1}, {b:2}); // expected: {a:1, b:2} (where the input a:1 object is modified)

{ a: 1 }

### zipObject(pairs) is broken

`zipObject([key, val, key2, val2, ..])` does not work anymore - it returns a fn expecting the vals array:

In [ ]:
_.zipObject(["key", "val"]); // expected => {key: "val"}

### Find on objects doesn't get the key parameter

In [3]:
lodash.find({key:"val"}, console.log.bind(console)) // => value, key, the whole object

val key { key: 'val' }


undefined

In [5]:
_.find(console.log.bind(console), {key:"val"}) // => only val :-(

val


undefined